# Schema: Document Type Definition (DTD)

Die Document Type Definition ermöglicht die Spezifikation von XML Dokumente und somit das Validieren solcher Dokumente. Man kann mit der DTD Schemas für XML Dokumente entwickeln. So kann man sich unter mehreren Parteien auf ein gemeinsames Vokabular einigen, und die Interoperabilität zwischen entwickelten Systeme ermöglichen oder erhöhen. In dieser Übung schauen wir uns die DTD etwas genauer in der Praxis an. Führen Sie zuerst den folgenden Codeblock aus und machen Sie dann der Reihe nach weiter. Beantworten Sie die Fragen (falls zutreffend). Zum Schluss, schreiben Sie eine eigene DTD und ein exemplarisches XML Dokument dafür. Stellen Sie sicher, dass das XML Dokument wohlgeformt und gültig ist.

In [1]:
import io
from lxml import etree as et

def isvalid(dtd, doc):
    print(et.DTD(io.StringIO(dtd)).validate(et.fromstring(doc)))
    
def exp(doc, path):
    print(et.fromstring(doc).xpath(path))

## Elemente

In [2]:
isvalid('<!ELEMENT discography EMPTY>', '<discography/>')

True


In [3]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums EMPTY>
"""

isvalid(dtd, """
<discography/>
""")

isvalid(dtd, """
<discography>
  <albums/>
</discography>
""")

# Warum ist dieses XML Dokument nicht gültig? Antwort: durch <!ELEMENT discography (albums)> wird nur ein element albums deklariert, da hier zwei albums auftreten ist das XML Dokument nicht gültig
isvalid(dtd, """
<discography>
  <albums/>
  <albums/>
</discography>
""")

False
True
False


In [4]:
dtd = """
<!ELEMENT discography (albums*)>
<!ELEMENT albums EMPTY>
"""

isvalid(dtd, """
<discography/>
""")

isvalid(dtd, """
<discography>
  <albums/>
</discography>
""")

# Warum ist dieses XML Dokument gültig? Antwort: durch das Sternchen bei der Deklarierung des Elements albums, siehe: <!ELEMENT discography (albums*)> sind Wiederholungen von null bis mehrmals möglich
isvalid(dtd, """
<discography>
  <albums/>
  <albums/>
</discography>
""")

True
True
True


In [5]:
dtd = """
<!ELEMENT discography (albums?)>
<!ELEMENT albums EMPTY>
"""

isvalid(dtd, """
<discography/>
""")

isvalid(dtd, """
<discography>
  <albums/>
</discography>
""")

# Warum ist dieses XML Dokument nicht gültig? Antwort: durch das Fragezeichen bei der Deklarierung des Elements albums, siehe: <!ELEMENT discography (albums?)> sind Wiederholungen von null bis 1 möglich. Da hier das Element zweimal auftaucht, ist das XML Dokument ungültig.
isvalid(dtd, """
<discography>
  <albums/>
  <albums/>
</discography>
""")

True
True
False


In [6]:
dtd = """
<!ELEMENT discography (albums+)>
<!ELEMENT albums EMPTY>
"""

# Warum ist dieses XML Dokument nicht gültig? Antwort: durch das Plus bei der Deklarierung des Elements albums, siehe: <!ELEMENT discography (albums+)> sind Wiederholungen von 1 bis mehrmals möglich. Da aber das Element albums nicht im Dokument auftaucht, ist es ungültig.
isvalid(dtd, """
<discography/>
""")

isvalid(dtd, """
<discography>
  <albums/>
</discography>
""")

isvalid(dtd, """
<discography>
  <albums/>
  <albums/>
</discography>
""")

False
True
True


In [7]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album+)>
<!ELEMENT album (title)>
<!ELEMENT title (#PCDATA)>
"""

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
    </album>
  </albums>
</discography>
""")

True


In [8]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album+)>
<!ELEMENT album (title, label)>
<!ELEMENT title (#PCDATA)>
<!ELEMENT label (#PCDATA)>
"""

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
      <label>Harvest, EMI</label>
    </album>
  </albums>
</discography>
""")

# Warum ist dieses XML Dokument nicht gültig? Antwort: die Elemente title und label sind in der Reihenfolge vertauscht.
isvalid(dtd, """
<discography>
  <albums>
    <album>
      <label>Harvest, EMI</label>
      <title>The Dark Side of the Moon</title>
    </album>
  </albums>
</discography>
""")

True
False


In [9]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album+)>
<!ELEMENT album (title, label)>
<!ELEMENT title (#PCDATA)>
"""

# Warum ist dieses XML Dokument nicht gültig? Antwort: für das Element label fehlt der Inhalt. Man müsste im dtd ergänzen: <!ELEMENT label (#PCDATA)>.
isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
      <label>Harvest, EMI</label>
    </album>
  </albums>
</discography>
""")

False


In [10]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album+)>
<!ELEMENT album (#PCDATA | title)*>
<!ELEMENT title (#PCDATA)>
"""

isvalid(dtd, """
<discography>
  <albums>
    <album>The Dark Side of the Moon</album>
  </albums>
</discography>
""")

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
    </album>
  </albums>
</discography>
""")

True
True


In [11]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album+)>
<!ELEMENT album (title | label)>
<!ELEMENT title (#PCDATA)>
<!ELEMENT label (#PCDATA)>
"""

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
    </album>
  </albums>
</discography>
""")

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <label>Harvest, EMI</label>
    </album>
  </albums>
</discography>
""")

# Warum ist dieses XML Dokument nicht gültig? Antwort: da eine Alternative bei album zwischen title und label besteht, kann nur eins von beiden auftreten.
isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
      <label>Harvest, EMI</label>
    </album>
  </albums>
</discography>
""")

True
True
False


In [12]:
isvalid("""
<!ELEMENT discography (albums)>
<!ELEMENT albums (album*)>
<!ELEMENT album (title, label, released)>
<!ELEMENT title (#PCDATA)>
<!ELEMENT label (#PCDATA)>
<!ELEMENT released (day, month, year)>
<!ELEMENT day (#PCDATA)>
<!ELEMENT month (#PCDATA)>
<!ELEMENT year (#PCDATA)>
""", """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
      <label>Harvest, EMI</label>
      <released>
        <day>16</day>
        <month>03</month>
        <year>1973</year>
      </released>
    </album>
  </albums>
</discography>
""")

True


In [13]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album*)>
<!ELEMENT album (title, label, released?)>
<!ELEMENT title (#PCDATA)>
<!ELEMENT label (#PCDATA)>
<!ELEMENT released ((day, month)?, year)>
<!ELEMENT day (#PCDATA)>
<!ELEMENT month (#PCDATA)>
<!ELEMENT year (#PCDATA)>
"""

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
      <label>Harvest, EMI</label>
      <released>
        <day>16</day>
        <month>03</month>
        <year>1973</year>
      </released>
    </album>
  </albums>
</discography>
""")

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
      <label>Harvest, EMI</label>
    </album>
  </albums>
</discography>
""")


isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
      <label>Harvest, EMI</label>
      <released>
        <year>1973</year>
      </released>
    </album>
  </albums>
</discography>
""")

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
      <label>Harvest, EMI</label>
    </album>
    <album>
      <title>The Wall</title>
      <label>Harvest, EMI</label>
      <released>
        <year>1979</year> 
      </released>
    </album>
  </albums>
</discography>
""")

True
True
True
True


## Attribute

In [14]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album*)>
<!ELEMENT album (title)>
<!ELEMENT title (#PCDATA)>
<!ATTLIST title released CDATA "1973">
"""

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title released="1973">The Dark Side of the Moon</title>
    </album>
  </albums>
</discography>
""")

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title released="1979">The Wall</title>
    </album>
  </albums>
</discography>
""")

True
True


In [15]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album*)>
<!ELEMENT album (title)>
<!ELEMENT title (#PCDATA)>
<!ATTLIST title released CDATA #REQUIRED> 
"""

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title released="1973">The Dark Side of the Moon</title>
    </album>
  </albums>
</discography>
""")

# Warum ist dieses XML Dokument nicht gültig? Antwort: da ein zwingendes Attribut deklariert worden ist durch #required und es in diesem Dokument nicht zu finden ist, ist das XML Dokument ungültig.
isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
    </album>
  </albums>
</discography>
""")

True
False


In [16]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album*)>
<!ELEMENT album (title)>
<!ELEMENT title (#PCDATA)>
<!ATTLIST title released CDATA #IMPLIED>
"""

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title released="1973">The Dark Side of the Moon</title>
    </album>
  </albums>
</discography>
""")

# Warum ist dieses XML Dokument gültig? Antwort: durch die Deklarierung des optionalen Attributs title released durch #IMPLIED muss es in diesem Dokument nicht vorkommen.
isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
    </album>
  </albums>
</discography>
""")

True
True


In [17]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album*)>
<!ELEMENT album (title)>
<!ELEMENT title (#PCDATA)>
<!ATTLIST title released CDATA #FIXED "1973">
"""

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title released="1973">The Dark Side of the Moon</title>
    </album>
  </albums>
</discography>
""")

# Warum ist dieses XML Dokument nicht gültig? Antwort: durch #FIXED "1973" kann hier das Attribut weggelassen werden oder muss mit 1973 festgelegt werden. Somit führt 1979 hier zu einem falschen Ergebnis.
isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title released="1979">The Wall</title>
    </album>
  </albums>
</discography>
""")

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title>The Wall</title>
    </album>
  </albums>
</discography>
""")

True
False
True


In [18]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album*)>
<!ELEMENT album (title)>
<!ELEMENT title (#PCDATA)>
<!ATTLIST title released (1973 | 1979) #REQUIRED>
"""

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title released="1973">The Dark Side of the Moon</title>
    </album>
  </albums>
</discography>
""")

# Warum ist dieses XML Dokument gültig? Antwort: durch das Sternchen im dtd bei album können mehrere Elemente album erscheinen. Auch ist das nötige #REQUIRED Attribut vorzufinden.
isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title released="1973">The Dark Side of the Moon</title>
    </album>
    <album>
      <title released="1979">The Wall</title>
    </album>
  </albums>
</discography>
""")

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title released="1982">The Wall</title>
    </album>
  </albums>
</discography>
""")

True
True
False


In [19]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album*)>
<!ELEMENT album (title)>
<!ELEMENT title (#PCDATA)>
<!ATTLIST title identifier ID #REQUIRED>
"""

isvalid(dtd, """
<discography>
  <albums>
    <album>
      <title identifier="p1">The Dark Side of the Moon</title>
    </album>
    <album>
      <title identifier="p2">The Wall</title>
    </album>
  </albums>
</discography>
""")

# Warum ist dieses XML Dokument nicht gültig? Antwort: durch die Deklarierung eines Attributs mit eindeutiger Id können nicht zwei gleiche Attribute mit selber ID existieren.
<discography>
  <albums>
    <album>
      <title identifier="p1">The Dark Side of the Moon</title>
    </album>
    <album>
      <title identifier="p1">The Wall</title>
    </album>
  </albums>
</discography>
""")

True
False


In [20]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album*)>
<!ELEMENT album EMPTY>
<!ATTLIST album title CDATA #REQUIRED>
<!ATTLIST album released CDATA #IMPLIED>
"""

isvalid(dtd, """
<discography>
  <albums>
    <album title="The Dark Side of the Moon" released="1973"/>
  </albums>
</discography>
""")

isvalid(dtd, """
<discography>
  <albums>
    <album title="The Dark Side of the Moon"/>
  </albums>
</discography>
""")

# Warum ist dieses XML Dokument nicht gültig? Antwort: Das #REQUIRED Attribut (album title)fehlt, das zwingend ist.
isvalid(dtd, """
<discography>
  <albums>
    <album released="1973"/>
  </albums>
</discography>
""")

True
True
False


In [21]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album*)>
<!ELEMENT album EMPTY>
<!ATTLIST album title CDATA #REQUIRED
                released CDATA #IMPLIED>
"""

isvalid(dtd, """
<discography>
  <albums>
    <album title="The Dark Side of the Moon" released="1973"/>
  </albums>
</discography>
""")

True


## Entitäten

In [22]:
dtd = """
<!ELEMENT discography (albums)>
<!ELEMENT albums (album*)>
<!ELEMENT album (#PCDATA)>
"""

doc = """
!DOCTYP<E discography [
<!ENTITY waters "Roger Waters">
]>
<discography>
  <albums>
    <album>&waters;</album>
  </albums>
</discography>
"""

isvalid(dtd, doc)

# Warum ergibt dies 'Roger Waters'? Antwort: da waters als Entität Roger Waters deklariert worden ist, ergibt die xpath abfrage Roger Waters
exp(doc, '/discography/albums/album/text()')

True
['Roger Waters']


## Namensräume

In [23]:
dtd = """
<!ELEMENT disc:discography (albs:albums)>
<!ELEMENT albs:albums (albs:album*)>
<!ELEMENT albs:album EMPTY>
<!ATTLIST disc:discography xmlns:disc CDATA #FIXED "http://discography.org">
<!ATTLIST disc:discography xmlns:albs CDATA #FIXED "http://albums.org">
<!ATTLIST albs:album title CDATA #REQUIRED>
<!ATTLIST albs:album released CDATA #REQUIRED>
"""

doc = """
<disc:discography xmlns:disc="http://discography.org" xmlns:albs="http://albums.org">
<albs:albums>
<albs:album title="The Dark Side of the Moon" released="1973"/>
</albs:albums>
</disc:discography>
"""

isvalid(dtd, doc)

True


Denken Sie sich nun ein eigenes XML Dokument aus und erstellen Sie dafür eine DTD.

In [58]:
dtd = """
<!ELEMENT buchsammlung (buch+)>
<!ELEMENT buch (name, autor)>
<!ELEMENT name (#PCDATA)>
<!ELEMENT autor EMPTY>
<!ATTLIST autor id ID #REQUIRED>
"""

doc = """
<!DOCTYPE buchsammlung [
<!ENTITY haus "Das leere Haus">
<!ENTITY blackwood "Blackwood">
]>
<buchsammlung>
<buch>
<name>&haus;</name>
<autor id="a1"></autor>
<!-- <autor id="a1">&blackwood;</autor> -->  #funktioniert nicht weil autor EMPTY
</buch>
<buch>
<name></name>
<autor id="a2"></autor>
</buch>
</buchsammlung>
"""
isvalid(dtd, doc)

True
